In [1]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/ProgettoDRV

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/ProgettoDRV


In [7]:
import pandas as pd
import numpy as np

# BITCOIN
bit = pd.read_csv('data/crypto/bitcoin.csv')
bit['Date'] = pd.to_datetime(bit["Date"])
bit = bit.set_index(['Date'])
bit = bit.asfreq("D", method="ffill")
bit = bit[bit.index >= pd.Timestamp("2019-09-01")]
bit = bit[bit.index <= pd.Timestamp("2022-03-10")]

# GOLD
gold = pd.read_csv("data/goldETC.csv")
gold.Date = pd.to_datetime(gold["Date"])
gold = gold[gold["Date"] >= pd.Timestamp("2019-09-01")]
gold = gold[gold["Date"] <= pd.Timestamp("2022-03-10")]
# Set date as index
gold = gold.set_index("Date")
gold = gold.asfreq("D", method="ffill")


# COVID
data = pd.read_csv('data/covid.csv')
# Converte la data
data['date'] = pd.to_datetime(data["date"])
# Select a subset of dataset's rows (select only total world data)
world_data = data[data["iso_code"] == "OWID_WRL"]
# Set date as index of the dataset
world_data = world_data.set_index(['date'])

wd = world_data[["new_cases", "new_deaths", "new_vaccinations"]]


# S&P500
sep = pd.read_csv("data/sep500.csv")
sep = sep.rename(columns={"Data":"Date"})
sep = sep.rename(columns={"Chiusura/Ultimo":"Close"})
sep = sep[["Date","Close","Massimo"]]
sep.Date = pd.to_datetime(sep.Date)
sep = sep.set_index("Date")

sep = sep[sep.index >= pd.Timestamp("2019-09-01")]
sep = sep[sep.index <= pd.Timestamp("2022-03-10")]

# BundGermania
bund = pd.read_csv("data/bund10ygermania.csv")
bund = bund.rename(columns={"Data":"Date"})
bund.Date = pd.to_datetime(bund.Date, dayfirst = True)
bund = bund.set_index("Date")
bund = bund.rename(columns={"Ultimo":"Close"})
bund = bund.rename(columns={"Massimo":"Max"})
bund = bund[["Close", "Max"]]


bund["Close"] = bund["Close"].astype(str).apply(lambda x: x.replace(',','.'))
bund["Max"] = bund["Max"].astype(str).apply(lambda x: x.replace(',','.'))

# Add all the missing days in the dataset
bund = bund.resample("D").mean()

bund = bund[bund.index >= pd.Timestamp("2019-09-01")]
bund = bund[bund.index <= pd.Timestamp("2022-03-10")]

# Fill missing values of the new added days
bund = bund.fillna(method='ffill')


In [9]:
gold= (gold - gold.min()) / (gold.max() - gold.min())
wd= (wd - wd.min()) / (wd.max() - wd.min())
bit= (bit - bit.min()) / (bit.max() - bit.min())
sep = (sep - sep.min()) / (sep.max() - sep.min())
bund = (bund - bund.min()) / (bund.max() - bund.min())

In [11]:
import plotly.graph_objects as go

import pandas as pd



# Create figure
fig = go.Figure()


fig.add_trace(
    go.Scatter(x=list(gold.index), y=list(gold.Close), name = "Gold ETC", marker=dict(color="gold")))


fig.add_trace(
    go.Scatter(x=list(wd.index), y=list(wd.new_cases), name = "New covid cases"))


fig.add_trace(
    go.Scatter(x=list(bit.index), y=list(bit.Close), name = "Bitcoin", marker=dict(color="#FFA15A")))

fig.add_trace(
    go.Scatter(x=list(bund.index), y=list(bund.Close), name = "Bund", marker=dict(color="royalblue")))

fig.add_trace(
    go.Scatter(x=list(sep.index), y=list(sep.Close), name = "S&P500", marker=dict(color="green")))

fig.add_annotation(x="2022-02-24", y=0.7,
            text="Russia invasion of Ucraina",
            showarrow=True,
            arrowhead=2,
            bordercolor="royalblue",
        borderwidth=2,
        borderpad=4,
        bgcolor="lightblue",
        opacity=0.8)

#fig.add_annotation(x="2020-12-17", y=0.4,
#            text="USA start vaccination",
#            showarrow=True,
#            arrowhead=2,
#            bordercolor="royalblue",
#        borderwidth=2,
#        borderpad=4,
#        bgcolor="lightblue",
#        opacity=0.8)


fig.add_annotation(x="2020-01-23", y=0.2,
            text="Start Covid-19 pandemic",
            showarrow=True,
            arrowhead=2,
            bordercolor="royalblue",
        borderwidth=2,
        borderpad=4,
        bgcolor="lightblue",
        opacity=0.8)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    # Modifica titolo
    title={
        'text': "Assets and daily covid-19 cases relationship",
        'y':0.90,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'}
)

fig.show()

fig.write_html("./grafici/gold_btc_covid.html")